In [1]:
import numpy as np
import pandas as pd

from nltk.tokenize import word_tokenize
import nltk
nltk.download('punkt')

from sklearn.linear_model import LogisticRegression
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer



from sklearn.metrics import classification_report, roc_auc_score
from sklearn.model_selection import train_test_split
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import MaxAbsScaler



[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


In [ ]:
# !pip install nltk

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [2]:
!unzip /content/drive/MyDrive/NLP/test.zip

Archive:  /content/drive/MyDrive/NLP/test.zip
  inflating: test_tin.csv            
  inflating: train_tin.csv           
  inflating: Тестовое задание по NLP от BST.docx  


In [3]:
train = pd.read_csv(r'/content/train_tin.csv', encoding='Windows-1251', sep=',')
test = pd.read_csv(r'/content/test_tin.csv', encoding='Windows-1251', sep=',')

In [ ]:
print(f'Размер датасета {train.shape[0]}')
print(f'Позитивных наблюдений {train[train.isPositive == 1].shape[0]}, негативных наблюдений {train[train.isPositive == 0].shape[0]}')

Размер датасета 3950
Позитивных наблюдений 1975, негативных наблюдений 1975


In [14]:
X_train, X_test, y_train, y_test = train_test_split(train.text, train.isPositive, random_state=13, shuffle=True)
X_train.reset_index(drop=True, inplace=True)
X_test.reset_index(drop=True, inplace=True)
y_train.reset_index(drop=True, inplace=True)
y_test.reset_index(drop=True, inplace=True)

Векторизуем наши данные с помощью CountVectorizer и TfidfVectorizer.
CountVectorizer делает простую вещь:

## CountVectorizer
Cтроит для каждого документа (каждой пришедшей ему строки) вектор размерности n, где n -- количество слов или n-грам во всём корпусе
заполняет каждый i-тый элемент количеством вхождений слова в данный документ

## TF-IDF
Эта мера, которая характеризует важность слова для конкретного текста. Рассчитывается следующим образом: для каждого слова из текста $d$ рассчитаем относительную частоту встречаемости в нем (Term Frequency):
$$
\text{TF}(t, d) = \frac{C(t | d)}{\sum\limits_{k \in d}C(k | d)},
$$
где $C(t | d)$ - число вхождений слова $t$ в текст $d$.

Также для каждого слова из текста $d$ рассчитаем обратную частоту встречаемости в корпусе текстов $D$ (Inverse Document Frequency):
$$
\text{IDF}(t, D) = \log\left(\frac{|D|}{|\{d_i \in D \mid t \in d_i\}|}\right)
$$
Логарифмирование здесь проводится с целью уменьшить масштаб весов, ибо зачастую в корпусах присутствует очень много текстов.

В итоге каждому слову $t$ из текста $d$ теперь можно присвоить вес
$$
\text{TF-IDF}(t, d, D) = \text{TF}(t, d) \times \text{IDF}(t, D)
$$
Интерпретировать данную формулу можно так: чем чаще данное слово встречается в данном тексте и чем реже в остальных, тем важнее оно для этого текста.

Отмечу, что в качестве TF и IDF можно использовать другие [определения](https://en.wikipedia.org/wiki/Tf%E2%80%93idf#Definition).

В целом можно было использовать любой понравившейся нам классифактор, я буду использовать логарфимическую регрессию (для ускорения процесса будут браться первые 200 итераций).

Ошибка будет выводиться с помощью confusion matrix.

ngram_range=(1, 1) — униграммы

In [ ]:
vec = CountVectorizer(ngram_range=(1, 1))
bow = vec.fit_transform(X_train)
bow_test = vec.transform(X_test)

scaler = MaxAbsScaler()
bow = scaler.fit_transform(bow)
bow_test = scaler.transform(bow_test)

In [ ]:
clf = LogisticRegression(max_iter=200, random_state=42)
clf.fit(bow, y_train)
pred = clf.predict(bow_test)
print(classification_report(y_test, pred))

              precision    recall  f1-score   support

           0       0.95      0.97      0.96       525
           1       0.97      0.94      0.95       463

    accuracy                           0.96       988
   macro avg       0.96      0.96      0.96       988
weighted avg       0.96      0.96      0.96       988



ngram_range=(3, 3) — триграммы

In [ ]:
vec = CountVectorizer(ngram_range=(3, 3))
bow = vec.fit_transform(X_train) 
bow_test = vec.transform(X_test)


scaler = MaxAbsScaler()
bow = scaler.fit_transform(bow)
bow_test = scaler.transform(bow_test)

In [ ]:
clf = LogisticRegression(max_iter=200, random_state=42)
clf.fit(bow, y_train)
pred_thrgramm = clf.predict(bow_test)
print(classification_report(y_test, pred_thrgramm))

              precision    recall  f1-score   support

           0       0.99      0.31      0.47       525
           1       0.56      1.00      0.72       463

    accuracy                           0.63       988
   macro avg       0.77      0.65      0.59       988
weighted avg       0.79      0.63      0.59       988



Для 3 грамм намного меньше f1-score, чем для 1 грамм.

In [ ]:
vec = TfidfVectorizer(ngram_range=(1, 1))
vec_train = vec.fit_transform(X_train)
vec_test = vec.transform(X_test)


scaler = MaxAbsScaler()
vec_train = scaler.fit_transform(vec_train)
vec_test = scaler.transform(vec_test)

In [ ]:
clf = LogisticRegression(max_iter=300, random_state=42)
clf.fit(vec_train, y_train)
pred_tfidf = clf.predict(vec_test)
print(classification_report(y_test, pred_tfidf))

              precision    recall  f1-score   support

           0       0.94      0.97      0.95       525
           1       0.97      0.93      0.95       463

    accuracy                           0.95       988
   macro avg       0.95      0.95      0.95       988
weighted avg       0.95      0.95      0.95       988



Раньше мы принимали пунктуацию за шум. Давайте посмотрим, что будет если не убирать ее.

In [ ]:
vec = CountVectorizer(ngram_range=(1, 1), tokenizer=word_tokenize)
bow = vec.fit_transform(X_train) 
bow_test = vec.transform(X_test)

scaler = MaxAbsScaler()
bow = scaler.fit_transform(bow)
bow_test = scaler.transform(bow_test)

In [ ]:
clf = LogisticRegression(max_iter=200, random_state=42)
clf.fit(bow, y_train)
pred = clf.predict(bow_test)
print(classification_report(y_test, pred))

              precision    recall  f1-score   support

           0       0.94      0.96      0.95       525
           1       0.96      0.94      0.95       463

    accuracy                           0.95       988
   macro avg       0.95      0.95      0.95       988
weighted avg       0.95      0.95      0.95       988



С пунктацией скор немного уменьшился. В некоторых задачах пунктуациях может многое сказать о тональности высказывания. Например, значок `)` может указывать о положительном отзыве.

Попробуем в качестве признаков использовать униграммы символов:

In [ ]:
vec = CountVectorizer(ngram_range=(1, 1), analyzer='char')
bow = vec.fit_transform(X_train) 
bow_test = vec.transform(X_test)

scaler = MaxAbsScaler()
bow = scaler.fit_transform(bow)
bow_test = scaler.transform(bow_test)

In [ ]:
clf = LogisticRegression(max_iter=200, random_state=42)
clf.fit(bow, y_train)
pred = clf.predict(bow_test)
print(classification_report(y_test, pred))

              precision    recall  f1-score   support

           0       0.89      0.69      0.77       525
           1       0.72      0.90      0.80       463

    accuracy                           0.79       988
   macro avg       0.80      0.79      0.79       988
weighted avg       0.81      0.79      0.79       988



### BERT

Попробуем осуществить следующую логику. Возьмем предобученную нейронку, подадим ей наши отзывы, она вернет нам числовое описание наших отзывов, мы попадим их нашему класссификатору.

In [5]:
! wget http://files.deeppavlov.ai/deeppavlov_data/bert/sentence_ru_cased_L-12_H-768_A-12_pt.tar.gz
!tar --gunzip --extract --verbose --file="sentence_ru_cased_L-12_H-768_A-12_pt.tar.gz"
!pip install deeppavlov
!pip install transformers

--2022-11-01 05:40:23--  http://files.deeppavlov.ai/deeppavlov_data/bert/sentence_ru_cased_L-12_H-768_A-12_pt.tar.gz
Resolving files.deeppavlov.ai (files.deeppavlov.ai)... 178.63.27.41
Connecting to files.deeppavlov.ai (files.deeppavlov.ai)|178.63.27.41|:80... connected.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: https://files.deeppavlov.ai/deeppavlov_data/bert/sentence_ru_cased_L-12_H-768_A-12_pt.tar.gz [following]
--2022-11-01 05:40:23--  https://files.deeppavlov.ai/deeppavlov_data/bert/sentence_ru_cased_L-12_H-768_A-12_pt.tar.gz
Connecting to files.deeppavlov.ai (files.deeppavlov.ai)|178.63.27.41|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 661614603 (631M) [application/octet-stream]
Saving to: ‘sentence_ru_cased_L-12_H-768_A-12_pt.tar.gz’

sentence_ru_cased_L 100%[===================>] 630.96M  19.7MB/s    in 34s     

2022-11-01 05:40:57 (18.8 MB/s) - ‘sentence_ru_cased_L-12_H-768_A-12_pt.tar.gz’ saved [661614603/661614603]

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 5.3 MB 8.0 MB/s 
     |████████████████████████████████| 7.6 MB 47.9 MB/s 
     |████████████████████████████████| 163 kB 72.3 MB/s 


In [6]:
from deeppavlov.core.common.file import read_json
from deeppavlov import build_model, configs
import torch

from string import punctuation
from nltk.corpus import stopwords
nltk.download('stopwords')
noise = stopwords.words('russian') + list(punctuation)
# bert
bert_config = read_json(configs.embedder.bert_embedder)
bert_config['metadata']['variables']['BERT_PATH'] = 'sentence_ru_cased_L-12_H-768_A-12_pt'

m = build_model(bert_config)


[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package perluniprops to /root/nltk_data...
[nltk_data]   Unzipping misc/perluniprops.zip.
[nltk_data] Downloading package nonbreaking_prefixes to
[nltk_data]     /root/nltk_data...
[nltk_data]   Unzipping corpora/nonbreaking_prefixes.zip.
Some weights of the model checkpoint at /content/sentence_ru_cased_L-12_H-768_A-12_pt were not used when initializing BertModel: ['cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.seq_relationship.bias', 'cls.predictions.decoder.weight', 'cls.seq_relationship.weight', 'cls.predictions.transform.dense.bias', 

In [7]:
list_name = [0] * len(X_train)
for i in range(len(list_name)):
  list_name[i] = X_train[i].translate(noise)

In [8]:
from tqdm import tqdm
res = []
error = []
with torch.no_grad():
  for j in tqdm(range((len(list_name)))):
    try:
      l_batch = list_name[j: (j+1)]
      tokens, token_embs, subtokens, subtoken_embs, sent_max_embs, sent_mean_embs, bert_pooler_outputs = m(l_batch)
      res.append(sent_mean_embs)
      torch.cuda.empty_cache()
    except:
      print(l_batch)
      error.append(j)

 15%|█▍        | 434/2963 [00:14<00:37, 67.99it/s]

['ДобрыйотденьменяотбытьнегоуженибылужебытьужотвотчатеотмобильногоотприложенияотБанкаотТинькоффотнаписалаотпоотповодуотсменыотадресаотпроживаниялиотаотточнееотпоинтересоваласьотвозможноотлиотизотвкладкиот?адреса»отубратьотнеактуальныйотадресоткогдаприотэтомотнаправилаотещеотиотскриншотлиотобведяоткраснымоттолиотчтоотследуетотскорректироватьдажеейотСотрудникотБанкаотВалентинаотспросилаотсменилаотлиотяотадресотпроживанияотиот?ушлаотпроверять»ужеотСтраннолиотчтоотсотрудникотпослеотзавешиванияотклиентаотнеотпишетотстандартноеот?спасибоотзаотожидание»отиотвсёотвотэтомотдухеототужеотНоотэтоотладнолиотдалееотпослеотмоегоотответаот?да»отнаотвопросотВалентиныот?удобноотлиотпринятьотзвонокотототбанкаотдляотизмененияотданных»лиотвотдиалоготвступаетотбылеслийотсотрудникот–отВикториялиоткотораяотдажеотНЕотПОЗДОРОВАВШИСЬотототпишетлиотчтоотейотнеобходимоотознакомитьсяотсотпредыдущейотперепискойужеотАотгдеотжеотобещанныйотзвонокотототБанкаейотНоотнаотэтомот?веселье»отнеотзаканчиваетсяужеотСпустяотпар

 27%|██▋       | 791/2963 [00:19<00:33, 65.08it/s]

['ЕслиотвыотдорожитеотсвоимиотнервамиотеслиотниотприоткакихотобстоятельствахотнеотимейтеотделотсоткредитнымиотпродуктамиотэтогоотбанкаменяВотТинькоффеслибанкеотуотменяотбылаотоформленаоткредитнаяоткартаотиотпотребительскийоткредитужеотКредитотяотоформилаотвотавгустелиотпоотграфикуотплатежейотя*должнаотбылаотвноситьотпоотнемуотдоужбылниотрубужеотопятьеслигоотчислаоткаждогоотмесяцалиотначинаяотс*сентябряужеКредитнойоткартойотгодотнеотпользоваласьотвообщелиотвотиюлеотэтогоотгодаотснялаотсотнееотбылниотнининиоттысужеотбытьотавгустаотполностьюотпогасилаотвсюотсумму*задолженностилиоткоторуюотнаоттототмоментотвиделаотвотсвоемотинтернетеслибанке*когдадолгвдругпроцентыдажеотиотбылаотполностьюотувереналиотчтоотпооткредиткеотяотбольшеотничегоотне*должнауже*былнибудьотсентябряотмнеотпоступаетотпервоеотзаотвсеотвремяотсмсеслисообщениеотбанкалиотчтоотуотменяотпросроченотплатежотпооткредитнойоткартеотнаотсуммуотбытьдонегоотрубужеотиотбудет*начисленотштрафужеотЯотниотразуотвотжизниотнеотдопускалаотпро

 35%|███▌      | 1040/2963 [00:23<00:40, 47.63it/s]

['ОтветилотнаотопросотбанкаотоботиспользованииотвкладовотиотполучилотвотконцеотпредложениеотнаписатьототзывотнаотэтомздесьникогдакудасейчасужедвахотьужеотАотпочемуотбыоти*нетей*Пользуюсьотбанкомот?Тинькофф»отнесколькоотлетужеотНачалосьотзнакомствоотсотнимотсотжеланияототкрытьотвклад*когдапроцентотбылотпривлекательнымдажеужеотДляототкрытияотвкладаот?пришлось»отзавестиотдебетовуюоткартуоттутзачемздесьодинкудалиоткоторуюотпривёзотмнеотдомойотвотудобноеотмнеотвремяоткурьеружеототПотомотрешилотпопробоватьотиспользоватьотданнуюоткартуотдляотоплатыотвотмагазинахлиотчемуотспособствовалаотпредложеннаяоткатегорияотсотповышенным*кэшбэкомот?Супермаркеты»отужеотОткрыл*дополнительнуюоткартуотженеотужеотКатегория*?Супермаркеты»отпотомотпропалалиотноотпривычкаотосталасьотужеотСпустяотнекотороеотвремя*открылоткредитнуюоткартуот?онизачемзачемонисейчасдвазачемсейчасникогдамойоб»откогдаопять*такиотизотмеркантильныхотсоображенийлиотдлянакопленияотмильотототдажеужеототВотрезультатеотбанк*вотнастоящийотмомен

 71%|███████   | 2096/2963 [00:45<00:12, 68.85it/s]

['ВотоктябреотбылнибытьужотгодаотпутешествовалотсотсемьейотпоотИзраилюужеотВзялотвотпрокатотвоткомпанииоттутподраздлянадосебеотавтомобильужеотИспользовалотвоткачестве*отрасчета*откредитнуюоткартуотТинькофф*отонибылабылаотонитебяразбылатебячтобнадотожеужеотСчитаю*отэту*откарту*оточеньотудачнойотдля*отпутешествийлиоттакоткакотпоотней*отповышенный*откешбекотнаотбилетыотиотбронироваие*ототелейотнаоттутбезбезчемтебячтобдляуже*ПутешествиеотудалосьужеотНоотвидимоотгдееслитоотвотИерусалимеотвсеоттаки*отнарушил*отправилаотдорожногоотдвижениялиотприпарковалотмашинуотв*отнеположенном*отместеуже**Вотитогеоттутподраздлянадосебеотоказаласьоточень*отхитрожужеужеужейоткомпаниейужеотЯотзабронировалотдля*отсемьи*мытотхотьникогдапочтиздесьсейчасотсебехотьсобоникогдаужеотвотаэропорту*отмнеотменеджер*отсказаллиотчто*оттакойотмашины*отнетужеотИотпредложила*самсейчасдругойобхотьэтомсейчасобнеесейчасотонитожетогдаужеотЧтоотменя*отнеотустроилоужеотВотитоге*отпосле*отдлительных*отпереговоровотнашли*мытотхотьник

100%|██████████| 2963/2963 [01:00<00:00, 48.93it/s]

[]


In [9]:
res_last = res[-1][:, :1000]
for j in tqdm(range(len(res) - 1)):
  res_last = np.concatenate((res[j][:, :1000], res_last))

100%|██████████| 2957/2957 [00:03<00:00, 921.69it/s]


In [29]:
clf = LogisticRegression(max_iter=200, random_state=42)
clf.fit(res_last, y_train.drop(error))
pred = clf.predict(res_last)
print(classification_report(y_train.drop(error), pred))

              precision    recall  f1-score   support

           0       0.76      0.28      0.41      1448
           1       0.57      0.92      0.70      1510

    accuracy                           0.60      2958
   macro avg       0.67      0.60      0.55      2958
weighted avg       0.66      0.60      0.56      2958



/usr/local/lib/python3.7/dist-packages/sklearn/linear_model/_logistic.py:818: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression


In [ ]:
# Как мы видим даже на тренировочной выборке результат намного ниже, чем в случай CountVectorizer.

Обратимся к Достоевскому. Это библиотека для анализа тональности текстов на русском языке. Модель обучалась на наборе данных RuSentiment.

In [30]:
!pip install dostoevsky

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 68 kB 4.0 MB/s 
  Using cached pybind11-2.10.1-py3-none-any.whl (216 kB)
  Created wheel for fasttext: filename=fasttext-0.9.2-cp37-cp37m-linux_x86_64.whl size=3161585 sha256=5bf3d612b9456be62d2fe2114e119a13f484e139570a6cfb58ba524ba94ed6c0
  Stored in directory: /root/.cache/pip/wheels/4e/ca/bf/b020d2be95f7641801a6597a29c8f4f19e38f9c02a345bab9b
Successfully built fasttext


In [31]:
!python -m dostoevsky download fasttext-social-network-model

In [32]:
from dostoevsky.tokenization import RegexTokenizer
from dostoevsky.models import FastTextSocialNetworkModel

tokenizer = RegexTokenizer()
model = FastTextSocialNetworkModel(tokenizer=tokenizer)

results = model.predict(X_test, k=2)

In [104]:
pred = []
for j in range(len(results)):
  if 'negative' in list(results[j].keys())[0]:
    pred.append(0)
  elif 'positive' in list(results[j].keys())[0]:
    pred.append(1)
  elif 'negative' in list(results[j].keys())[1]:
    pred.append(0)
  elif 'positive' in list(results[j].keys())[1]:
    pred.append(1)
  # выбор
  elif 'neutral' in list(results[j].keys())[0] and 'skip' in list(results[j].keys())[1]:
    pred.append(1)
  elif 'skip' in list(results[j].keys())[0] and 'neutral' in list(results[j].keys())[1]:
    pred.append(1)
  elif 'speech' in list(results[j].keys())[0] and 'neutral' in list(results[j].keys())[1]:
    pred.append(1)
  elif 'neutral' in list(results[j].keys())[0] and 'speech' in list(results[j].keys())[1]:
    pred.append(1)
  elif 'speech' in list(results[j].keys())[0] and 'skip' in list(results[j].keys())[1]:
    pred.append(1)
  elif 'skip' in list(results[j].keys())[0] and 'speech' in list(results[j].keys())[1]:
    pred.append(1)

In [108]:
print(classification_report(y_test, pred))

              precision    recall  f1-score   support

           0       0.61      0.96      0.75       525
           1       0.88      0.30      0.45       463

    accuracy                           0.65       988
   macro avg       0.75      0.63      0.60       988
weighted avg       0.74      0.65      0.61       988



In [ ]:
# Можно выбирать конфигурации того, как мы кодируем нейтральность, скип и разговорный.